In [1]:
# 랭스미스 설정
from dotenv import load_dotenv

load_dotenv('./../.env')

True

In [2]:
# 주식관련 뉴스페이지 주소 리스트

urls = []
urls.append('https://economictimes.indiatimes.com/markets/stocks/news')
urls.append('https://www.livemint.com/latest-news')
urls.append('https://www.livemint.com/latest-news/page-2')
urls.append('https://www.livemint.com/latest-news/page-3')
urls.append('https://www.moneycontrol.com/')

print(urls)

['https://economictimes.indiatimes.com/markets/stocks/news', 'https://www.livemint.com/latest-news', 'https://www.livemint.com/latest-news/page-2', 'https://www.livemint.com/latest-news/page-3', 'https://www.moneycontrol.com/']


In [3]:
# 마켓 뉴스 페이지를 로드한다.
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(web_paths=urls)

loader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
# 지연 로드
docs = []
async for doc in loader.alazy_load():
    docs.append(doc)

docs

Fetching pages:   0%|          | 0/5 [00:00<?, ?it/s]

Fetching pages: 100%|##########| 5/5 [00:00<00:00, 23.87it/s]


[Document(metadata={'source': 'https://economictimes.indiatimes.com/markets/stocks/news', 'title': 'Stocks in News Today - Latest News on Stocks, Stock in News | The Economic Times', 'description': 'Stocks in News - Find the latest Stocks in News on The Economic Times. Get Stocks Analysis, News on Stocks and more.', 'language': 'en'}, page_content='Stocks in News Today - Latest News on Stocks, Stock in News | The Economic TimesBenchmarks Nifty23,031.40-13.85FEATURED FUNDS★★★★★HSBC Large Cap Fund Direct-Growth5Y Return14.97 %\n                Invest NowFEATURED FUNDS★★★★★Canara Robeco ELSS Tax Saver Regular - Growth5Y Return16.67 %\n                Invest NowEnglish EditionEnglish Editionहिन्दीગુજરાતીमराठीবাংলাಕನ್ನಡമലയാളംதமிழ்తెలుగు | 14 February, 2025, 03:03 AM IST | Today\'s ePaper\n            \t\t\t        My Watchlist\n                            SubscribeSign InHomeBUDGET\'25ETPrimeMarketsMarket DataNewsIndustryRisePoliticsWealthMFTechCareersOpinionNRIPanacheVideosMore MenuStocksN

In [5]:
def format_docs(docs):
    return "\n\n".join([x.page_content for x in docs])

In [6]:
# 리스트를 하나의 문자열로 이어붙인다 (\n\n으로 구분)
context = format_docs(docs)
context

'Stocks in News Today - Latest News on Stocks, Stock in News | The Economic TimesBenchmarks Nifty23,031.40-13.85FEATURED FUNDS★★★★★HSBC Large Cap Fund Direct-Growth5Y Return14.97 %\n                Invest NowFEATURED FUNDS★★★★★Canara Robeco ELSS Tax Saver Regular - Growth5Y Return16.67 %\n                Invest NowEnglish EditionEnglish Editionहिन्दीગુજરાતીमराठीবাংলাಕನ್ನಡമലയാളംதமிழ்తెలుగు | 14 February, 2025, 03:03 AM IST | Today\'s ePaper\n            \t\t\t        My Watchlist\n                            SubscribeSign InHomeBUDGET\'25ETPrimeMarketsMarket DataNewsIndustryRisePoliticsWealthMFTechCareersOpinionNRIPanacheVideosMore MenuStocksNewsLive BlogStock Live BlogEarningsPodcastMarket ClassroomDons of Dalal StreetRecosStock Reports PlusNewMy ScreenerCandlestick ScreenerStock ScreenerStock WatchMarket CalendarStock Price QuotesOptionsIPOs/FPOsExpert ViewsInvestment IdeasCommoditiesViewsNewsOthersMentha OilPrecious MetalsGold MGoldSilverGold PetalSilver MicroSilver MGold GuineaOil &

In [7]:
# 쓸데없는 \n과 \t 뭉치를 전처리한다
import re

def text_clean(text):
    text = re.sub(r'\n\n+', '\n\n', text)             # 2개 초과의 \n을 두개로 줄인다
    text = re.sub(r'\t+', '\t', text)               # 1개 초과의 \t를 하나로 줄인다
    text = re.sub(r'\s+', ' ', text)                # 1개 초과의 공백을 하나로 줄인다
    return text

In [8]:
context = text_clean(context)

print(context)

Stocks in News Today - Latest News on Stocks, Stock in News | The Economic TimesBenchmarks Nifty23,031.40-13.85FEATURED FUNDS★★★★★HSBC Large Cap Fund Direct-Growth5Y Return14.97 % Invest NowFEATURED FUNDS★★★★★Canara Robeco ELSS Tax Saver Regular - Growth5Y Return16.67 % Invest NowEnglish EditionEnglish Editionहिन्दीગુજરાતીमराठीবাংলাಕನ್ನಡമലയാളംதமிழ்తెలుగు | 14 February, 2025, 03:03 AM IST | Today's ePaper My Watchlist SubscribeSign InHomeBUDGET'25ETPrimeMarketsMarket DataNewsIndustryRisePoliticsWealthMFTechCareersOpinionNRIPanacheVideosMore MenuStocksNewsLive BlogStock Live BlogEarningsPodcastMarket ClassroomDons of Dalal StreetRecosStock Reports PlusNewMy ScreenerCandlestick ScreenerStock ScreenerStock WatchMarket CalendarStock Price QuotesOptionsIPOs/FPOsExpert ViewsInvestment IdeasCommoditiesViewsNewsOthersMentha OilPrecious MetalsGold MGoldSilverGold PetalSilver MicroSilver MGold GuineaOil & EnergyNatural GasCrude OilCrude Oil MiniBase MetalsAluminiumZinc MiniLead MiniCopperZincNick

In [9]:
from my_scripts import my_llm               # my_scripts 폴더 안에 my_llm.py 파일을 가져온다

response = my_llm.ask_llm(context, "Summarize this")

# 현재 모델은 시작과 끝부분만 주의깊게 보기 때문에, 긴 텍스트는 중간 내용이 망각되는 경우가 발생한다. => 작은 덩어리로 쪼개야한다 (chunks) => 청크별로 받은 답변을 합친다
print(response)

The article provides a news update, but the content seems to be more about Donald Trump and his meeting with Prime Minister Narendra Modi on February 14, 2025.

To provide a summary:

Donald Trump is reportedly planning to make some changes to India's tariffs during his meeting with PM Modi. The Indian government has expressed its intention to increase tariffs if any trade agreements are signed between the two countries.


In [10]:
# 웹에서 가져와서 -> 전처리하고 -> 영어로 번역하고 -> 청크로 나누고 -> llm에 물어봐서 답변받고 -> 각 청크에 대한 답변을 한글로 번역하고 -> 합쳐서 출력

In [11]:
# 청크로 나누기 함수
# 오버랩으로 곂쳐야 중요한 문자열이 잘려없어지는 것을 방지할 수 있다.
def chunk_text(text, chunk_size, overlap=100):
    chunks = []
    for i in range(0, len(text), chunk_size - overlap):
        chunks.append(text[i:i + chunk_size])
    return chunks

In [12]:
chunks = chunk_text(context, 5000)

In [13]:
chunks

['Stocks in News Today - Latest News on Stocks, Stock in News | The Economic TimesBenchmarks Nifty23,031.40-13.85FEATURED FUNDS★★★★★HSBC Large Cap Fund Direct-Growth5Y Return14.97 % Invest NowFEATURED FUNDS★★★★★Canara Robeco ELSS Tax Saver Regular - Growth5Y Return16.67 % Invest NowEnglish EditionEnglish Editionहिन्दीગુજરાતીमराठीবাংলাಕನ್ನಡമലയാളംதமிழ்తెలుగు | 14 February, 2025, 03:03 AM IST | Today\'s ePaper My Watchlist SubscribeSign InHomeBUDGET\'25ETPrimeMarketsMarket DataNewsIndustryRisePoliticsWealthMFTechCareersOpinionNRIPanacheVideosMore MenuStocksNewsLive BlogStock Live BlogEarningsPodcastMarket ClassroomDons of Dalal StreetRecosStock Reports PlusNewMy ScreenerCandlestick ScreenerStock ScreenerStock WatchMarket CalendarStock Price QuotesOptionsIPOs/FPOsExpert ViewsInvestment IdeasCommoditiesViewsNewsOthersMentha OilPrecious MetalsGold MGoldSilverGold PetalSilver MicroSilver MGold GuineaOil & EnergyNatural GasCrude OilCrude Oil MiniBase MetalsAluminiumZinc MiniLead MiniCopperZinc

In [14]:
# 적절한 답변이 나올때까지 question(프롬프트)를 변경한다
question = "Extract stock market news from the given text."

chunk_summary = []
for chunk in chunks:
    response = my_llm.ask_llm(chunk, question)
    chunk_summary.append(response)

In [15]:
for chunk in chunk_summary:
    print(chunk)
    print("\n\n")

Here are the extracted stock market news:

* Robinhood has gained a strong position in the crypto trading market with its commission-free model, attracting price-conscious customers and rivals such as Coinbase.
* Analysts predict continued momentum in crypto prices and market share growth, driven by retail traders' confidence.
* The Indian market closed nearly flat on Thursday due to easing domestic inflation being offset by U.S. inflation concerns and trade tensions.
* Kotak Mahindra Bank Share Price: 1972.8028.85 (1.49%)
* Nasdaq surges over 1% after producer prices data; Tesla jumps 5%, Nvidia 3%
* Shriram Finance Ltd (SFL) raises USD 500mn through ECB to fund business growth.
* Rashmi Saluja ceases to be director of Religare as shareholders shoot down her re-appointment.



Here are the stock market news extracted from the given text:

* Paytm Money pays ₹45.50 lakh to settle Sebi violation case.
* One97 Communications shares price increases by 1.35% to ₹755.40 (based on a share pr

In [16]:
# 청크 리스트를 하나의 문자열로 합친다
summary_data = "\n\n".join(chunk_summary)

print(summary_data)

Here are the extracted stock market news:

* Robinhood has gained a strong position in the crypto trading market with its commission-free model, attracting price-conscious customers and rivals such as Coinbase.
* Analysts predict continued momentum in crypto prices and market share growth, driven by retail traders' confidence.
* The Indian market closed nearly flat on Thursday due to easing domestic inflation being offset by U.S. inflation concerns and trade tensions.
* Kotak Mahindra Bank Share Price: 1972.8028.85 (1.49%)
* Nasdaq surges over 1% after producer prices data; Tesla jumps 5%, Nvidia 3%
* Shriram Finance Ltd (SFL) raises USD 500mn through ECB to fund business growth.
* Rashmi Saluja ceases to be director of Religare as shareholders shoot down her re-appointment.

Here are the stock market news extracted from the given text:

* Paytm Money pays ₹45.50 lakh to settle Sebi violation case.
* One97 Communications shares price increases by 1.35% to ₹755.40 (based on a share pric

In [17]:
question = "Write a detailed market news report in markdown format. Think carefully then write the report."

response = my_llm.ask_llm(summary_data, question)

In [18]:
# UTF-8 파일로 출력한다

import os

# 디렉토리 먼저 만들고
os.makedirs("data", exist_ok=True)

# 파일 생성
with open("data/report.md", "w", encoding="utf-8") as f:
    f.write(response)


In [19]:
import os

# 디렉토리 먼저 만들고
os.makedirs("data", exist_ok=True)

# 파일 생성
with open("data/summary.md", "w", encoding="utf-8") as f:
    f.write(summary_data)

In [25]:
from deep_translator import GoogleTranslator

translator_en_to_ko = GoogleTranslator(target='ko')                         # 언어 감지
# translator_en_to_ko = GoogleTranslator(source='en', target='ko')         # 영어를 한국어로 번역하고 다른 언어가 섞이면 안됨

lst_chunk_ko = []
for chunk in chunk_summary:
    lst_chunk_ko.append(translator_en_to_ko.translate(chunk))

str_chunk_ko = "\n\n".join(lst_chunk_ko)

# 디렉토리 먼저 만들고
os.makedirs("data", exist_ok=True)

# 파일 생성
with open("data/summary_ko.md", "w", encoding="utf-8") as f:
    f.write(str_chunk_ko)

In [26]:
from deep_translator import GoogleTranslator

translator = GoogleTranslator(source='en', target='ko')                  # 영어를 한국어로 번역 (다른 언어의 단어가 섞이면 안됨)
text = "India's new tax policy is impacting व्यापारियों heavily."
# व्यापारियों : 상인들, 거래자들

translated_text = translator.translate(text)
print(translated_text)


인도의 새로운 세금 정책은 경제에 큰 영향을 미치고 있습니다.


In [27]:
from deep_translator import GoogleTranslator

translator = GoogleTranslator(target='ko')                              # 언어 감지하여 모두 한국어로 번역
text = "India's new tax policy is impacting व्यापारियों heavily."
# व्यापारियों : 상인들, 거래자들

translated_text = translator.translate(text)
print(translated_text)

인도의 새로운 세금 정책은 거래자들에게 큰 영향을 미치고 있습니다.
